In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [49]:
data = pd.read_csv("wine.csv")
data

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,3,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,3,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,3,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,3,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [50]:
data.columns.tolist()

['Wine',
 'Alcohol',
 'Malic.acid',
 'Ash',
 'Acl',
 'Mg',
 'Phenols',
 'Flavanoids',
 'Nonflavanoid.phenols',
 'Proanth',
 'Color.int',
 'Hue',
 'OD',
 'Proline']

In [51]:
X = data.drop("Wine", axis = 1)
y = data['Wine']

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0,stratify=y)

In [53]:
from sklearn.preprocessing import StandardScaler


sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [54]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE

lr = LogisticRegression()

rfe = RFE(estimator=lr, n_features_to_select=5, step=1)
rfe.fit(X_train_std, y_train)

X_train_sub = rfe.transform(X_train)

C:\Users\Admin\AppData\Roaming\Python\Python314\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but RFE was fitted without feature names
  warnings.warn(


In [55]:
rfe.support_

array([ True, False, False, False, False, False,  True, False, False,
        True,  True, False,  True])

In [56]:
data.columns[1:][rfe.support_]

Index(['Alcohol', 'Flavanoids', 'Color.int', 'Hue', 'Proline'], dtype='object')

In [57]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline


pipe = make_pipeline(RFE(estimator=lr, step=1),
                     KNeighborsClassifier())


parameters = {'rfe__n_features_to_select': range(1, 13), 
              'kneighborsclassifier__n_neighbors': range(1, 10) }

grid = GridSearchCV(pipe, param_grid=parameters, cv=10, n_jobs=-1)
grid.fit(X_train_std, y_train)

print('Best params:', grid.best_params_)
print('Best accuracy:', grid.best_score_)

Best params: {'kneighborsclassifier__n_neighbors': 9, 'rfe__n_features_to_select': 8}
Best accuracy: 0.9839743589743589


In [59]:
grid.best_estimator_.score(X_test_std, y_test)

1.0

In [61]:
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(X_train_std, y_train)
knn.score(X_test_std, y_test)

0.9629629629629629